In [53]:
library("geneplotter")
library("ggplot2")
library("vsn")
library("genefilter")
library("DESeq2")
getwd()

[1] "/home/gstupp/projects/Wolan/wolan/Velos_AW_20170831"

In [54]:
meta_path = "metadata.csv"
sampleTable = read.csv(meta_path, row.names = 1)
sampleTable$biological=factor(sampleTable$biological)
sampleTable$technical=factor(sampleTable$technical)
sampleTable$sample_type=factor(sampleTable$sample_type)
sampleTable$sample_type2=factor(sampleTable$sample_type2)
sampleTable$sample_type <- relevel(sampleTable$sample_type, ref="unenriched")
sampleTable$sample_type2 <- relevel(sampleTable$sample_type2, ref="unenriched")
sampleTable

,biological,technical,unenriched,probe,sample_type,sample_type2,sample_name,path
Neg1rep1,1,1,FALSE,FALSE,neg,probe,neg1,data/Neg1rep1/DTASelect-filter.txt
Neg1rep2,1,2,FALSE,FALSE,neg,probe,neg1,data/Neg1rep2/DTASelect-filter.txt
Neg1rep3,1,3,FALSE,FALSE,neg,probe,neg1,data/Neg1rep3/DTASelect-filter.txt
Nov_unenriched1rep1,1,1,TRUE,FALSE,unenriched,unenriched,unenriched1,data/Nov_unenriched1rep1/DTASelect-filter.txt
Nov_unenriched1rep2,1,2,TRUE,FALSE,unenriched,unenriched,unenriched1,data/Nov_unenriched1rep2/DTASelect-filter.txt
Nov_unenriched1rep3,1,3,TRUE,FALSE,unenriched,unenriched,unenriched1,data/Nov_unenriched1rep3/DTASelect-filter.txt
Nov_unenriched2rep1,2,1,TRUE,FALSE,unenriched,unenriched,unenriched2,data/Nov_unenriched2rep1/DTASelect-filter.txt
Nov_unenriched2rep2,2,2,TRUE,FALSE,unenriched,unenriched,unenriched2,data/Nov_unenriched2rep2/DTASelect-filter.txt
Nov_unenriched2rep3,2,3,TRUE,FALSE,unenriched,unenriched,unenriched2,data/Nov_unenriched2rep3/DTASelect-filter.txt
Nov_unenriched3rep1,3,1,TRUE,FALSE,unenriched,unenriched,unenriched3,data/Nov_unenriched3rep1/DTASelect-filter.txt


In [55]:
countData = read.csv("out/df_filt.csv", row.names = 1)
locusTable = read.csv("out/locus_df.csv", row.names = 1)
sampleTable = sampleTable[colnames(countData),]

In [56]:
dds = DESeqDataSetFromMatrix(countData, colData = sampleTable, ~ sample_type + biological)

## collapse replicates
dds = collapseReplicates(dds, dds$sample_name)
sampleTable = colData(dds)

In [57]:
head(counts(dds))

,neg1,unenriched1,unenriched2,unenriched3,vk1,vk2,vk3
86769,0,18,20,21,0,0,0
202956,3,20,23,27,26,21,23
209005,25,39,35,40,36,42,22
318873,3,23,28,30,19,19,17
374028,0,20,16,18,0,0,0
496339,0,33,36,29,0,0,0


In [58]:
# subtract the neg samples
countData = counts(dds)
countData[,"vk1"] = countData[,"vk1"] - countData[,"neg1"]
countData[,"vk2"] = countData[,"vk2"] - countData[,"neg1"]
countData[,"vk3"] = countData[,"vk3"] - countData[,"neg1"]
countData[countData<0] = 0
head(countData)

,neg1,unenriched1,unenriched2,unenriched3,vk1,vk2,vk3
86769,0,18,20,21,0,0,0
202956,3,20,23,27,23,18,20
209005,25,39,35,40,11,17,0
318873,3,23,28,30,16,16,14
374028,0,20,16,18,0,0,0
496339,0,33,36,29,0,0,0


In [59]:
### get rid of the neg samples
non_neg = rownames(sampleTable[sampleTable$sample_type != "neg",])
countData = countData[,non_neg]
sampleTable = sampleTable[non_neg,]

In [60]:
head(countData)

,unenriched1,unenriched2,unenriched3,vk1,vk2,vk3
86769,18,20,21,0,0,0
202956,20,23,27,23,18,20
209005,39,35,40,11,17,0
318873,23,28,30,16,16,14
374028,20,16,18,0,0,0
496339,33,36,29,0,0,0


In [61]:
sampleTable

DataFrame with 6 rows and 8 columns
            biological technical unenriched     probe sample_type sample_type2
              <factor>  <factor>  <logical> <logical>    <factor>     <factor>
unenriched1          1         1       TRUE     FALSE  unenriched   unenriched
unenriched2          2         1       TRUE     FALSE  unenriched   unenriched
unenriched3          3         1       TRUE     FALSE  unenriched   unenriched
vk1                  1         1      FALSE      TRUE       probe        probe
vk2                  2         1      FALSE      TRUE       probe        probe
vk3                  3         1      FALSE      TRUE       probe        probe
            sample_name                                          path
               <factor>                                      <factor>
unenriched1 unenriched1 data/Nov_unenriched1rep1/DTASelect-filter.txt
unenriched2 unenriched2 data/Nov_unenriched2rep1/DTASelect-filter.txt
unenriched3 unenriched3 data/Nov_unenriched3rep1/DTA

In [62]:
dds = DESeqDataSetFromMatrix(countData, colData = sampleTable, ~ sample_type + biological)

converting counts to integer mode
factor levels were dropped which had no samples


In [63]:
dds = DESeq(dds)

estimating size factors
estimating dispersions
gene-wise dispersion estimates
mean-dispersion relationship
final dispersion estimates
fitting model and testing


In [64]:
#mcols(dds)<-cbind(mcols(dds),locusTable)
save(dds, file = "out/dds")

In [65]:
res = results(dds, contrast=c("sample_type","probe","unenriched"))
summary(res)


out of 9237 with nonzero total read count
adjusted p-value < 0.1
LFC > 0 (up)     : 316, 3.4% 
LFC < 0 (down)   : 2611, 28% 
outliers [1]     : 0, 0% 
low counts [2]   : 167, 1.8% 
(mean count < 2)
[1] see 'cooksCutoff' argument of ?results
[2] see 'independentFiltering' argument of ?results



In [66]:
res = cbind(res, locusTable)
save(res, file = "out/res_probe_unenriched")

In [67]:
resOrdered <- res[order(res$padj),]
write.csv(as.data.frame(resOrdered),file="out/probe_unenriched_deseq_results.csv")